In [2]:
!pip install tensorflow_datasets

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/5.3 MB ? eta -:--:--
    --------------------------------------- 0.1/5.3 MB 2.6 MB/s eta 0:00:02
   -- ------------------------------------- 0.3/5.3 MB 3.8 MB/s eta 0:00:02
   ----- ---------------------------------- 0.7/5.3 MB 5.4 MB/s eta 0:00:01
   --------- ------------------------------ 1.3/5.3 MB 7.3 MB/s eta 0:00:01
   ------------- -------------------------- 1.8/5.3 MB 8.2 MB/s eta 0:00:01
   ------------------ --------------------- 2.4/5.3 MB 9.1 MB/s eta 0:00:01
   ----------------------- ---------------- 3.1/5.3 MB 9.8 MB/s eta 0:00:01
   ------------------------- -------------- 3.4/5.3 MB 9.5 MB/s eta 0:00:01
   ----------------------------- ---------- 3.9/5.3 MB 9.6 MB/s eta 0:00:01
   ---------------------------------- ----- 4.6/5.3 MB 10.0 MB/s eta 0:00:01
   ------------------------------------- -- 4.9/5.3 MB 10.2 MB/s eta

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.32.0 requires protobuf<5,>=3.20, but you have protobuf 5.28.3 which is incompatible.
tensorflow-intel 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.28.3 which is incompatible.


In [3]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf

tfds.disable_progress_bar()

In [5]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])

In [6]:
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

Dataset imdb_reviews downloaded and prepared to C:\Users\halur\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [7]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
label:  0


In [8]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])


texts:  [b'I\'ll be honest. I got this movie so I could make fun of it. I mean, come on, "Hood of the Living Dead"? What other reaction could I have? The thing is, though, the movie (and its makers) decided that it wasn\'t going to be made fun of. Instead, it was going to try its best to be a good movie.<br /><br />And you know what? It came awfully close. A little less cheese in the incidental music, a little more professionalism in the photography, the acting, the incidentals (like the props--love the Best Buy bag)...well, it\'s not a classic of the zombie movie genre, but it\'s still a pretty neat little movie on its own. And the acting, writing and pacing are all surprisingly better than I would have expected. There\'s even some decent humor, as two of our leads debate how to decide if a dead zombie is really dead.<br /><br />If you can overlook the low budget (which leaves its fingerprints in everything, alas) and the almost constant profanity, this can be a pretty fun time at the

In [9]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [11]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

In [15]:
encoded_example=encoder(example)[:3].numpy()
encoded_example

array([[527,  28,   1, ...,   0,   0,   0],
       [ 41,   4, 878, ...,   0,   0,   0],
       [  4, 548,  17, ...,   0,   0,   0]], dtype=int64)

In [16]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b'I\'ll be honest. I got this movie so I could make fun of it. I mean, come on, "Hood of the Living Dead"? What other reaction could I have? The thing is, though, the movie (and its makers) decided that it wasn\'t going to be made fun of. Instead, it was going to try its best to be a good movie.<br /><br />And you know what? It came awfully close. A little less cheese in the incidental music, a little more professionalism in the photography, the acting, the incidentals (like the props--love the Best Buy bag)...well, it\'s not a classic of the zombie movie genre, but it\'s still a pretty neat little movie on its own. And the acting, writing and pacing are all surprisingly better than I would have expected. There\'s even some decent humor, as two of our leads debate how to decide if a dead zombie is really dead.<br /><br />If you can overlook the low budget (which leaves its fingerprints in everything, alas) and the almost constant profanity, this can be a pretty fun time at t

In [17]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [19]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True]


In [33]:
# predict on a sample text without padding.

sample_text = ('The movie was cool. The animation and the graphics'
               'were out of this world. I would recommend this movie.')

predictions = model.predict(np.array([sample_text]))
print(predictions[0])

ValueError: Invalid dtype: str3296

In [32]:
# predict on a sample text with padding

padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

ValueError: Invalid dtype: str256000